In [16]:
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import pandas as pd
import json
from liturgy.scraper import scrape_liturgies, scrape_message_ids, read_cookie
from liturgy.preprocessor import preprocess
from liturgy.ai_parser import query_ai


In [66]:
RERUN_GET_MSG_IDS = False
RERUN_SCRAPE = False
RERUN_AI = True

In [49]:

auth_cookie = read_cookie()
if RERUN_GET_MSG_IDS:
    scrape_message_ids(auth_cookie)
if RERUN_SCRAPE:
    scrape_liturgies(auth_cookie)

In [50]:
preprocess()

Parsing 154 files


In [51]:
df = pd.read_json("./data/output/zangdiensten.json")

In [52]:
from liturgy.regex_parser import Dienst
df["songs_regex"] = df.apply(lambda row:Dienst(row.liturgy, str(row.date)).liederen, axis=1)
df

Dienst with many semicolons found


,date,liturgy,file,full_content,songs_regex
0,2024-09-01,Even stil worden aan het begin van de dienst: ...,./data//raw/service_268190.html,Ruth (II): Pay-it-forward van genade: iets doe...,"[lied: 'samen in de naam van jezus', luisterli..."
1,2024-12-08,We beginnen met het 'Kaarsenlied' en het aanst...,./data//raw/service_358214.html,Van trots naar nederig\nDe kerkdienst wordt ge...,[we beginnen met het 'kaarsenlied' en het aans...
2,2024-01-07,Welkom\n Lied: Onze hulp en onze verwachting (...,./data//raw/service_258121.html,De kerkdienst wordt gehouden in de Immanuelker...,"[lied: onze hulp en onze verwachting (sela), l..."
3,2023-12-23,Dankdienst voorafgaand aan de uitvaart van zr....,./data//raw/service_301056.html,Dankdienst voorafgaand aan de uitvaart van zr....,[]
4,2023-01-15,"Opening van de dienst\nOpeningspsalm: 92:1,5,6...",./data//raw/service_230296.html,Het leven van Jezus: de eerste genezingen.\nDe...,"[lied: father we love you (nlb 875), gebed voo..."
...,...,...,...,...,...
149,2024-01-14,"Opening van de dienst\n Lied: \nJuicht, want J...",./data//raw/service_258122.html,De leeuw maar ook het Lam! - Het getuigenis va...,"[lied: juicht want jezus is heer!, lied: uw ..."
150,2024-08-11,Votum en groet (sela)\n zingen: Dit is morge...,./data//raw/service_268185.html,De kerkdienst wordt gehouden in de Immanuelker...,[zingen: dit is morgen als eens de eerste (nl...
151,2023-01-08,"Opening van de dienst\nLied: Jezus, wij verhog...",./data//raw/service_230295.html,Doop van Floor van Meeuwen - Drie Koningen: de...,"[lied: jezus wij verhogen u (opw. 237), lied: ..."
152,2024-03-29,Opening van de dienst\n Lied: Uw genade is mij...,./data//raw/service_268150.html,Met vallen en opstaan - met Petrus op weg naar...,"[lied: uw genade is mij genoeg (opw. 614), lui..."


In [ ]:
if RERUN_AI:
    query_ai(df)

94it [06:39,  5.52s/it]

In [69]:
with open("./data/raw/api_responses.json", 'r') as f:
    responses = json.loads(f.read())

ai_summaries = pd.DataFrame([(key, value.split("\n")) for key, value in responses.items()], columns=["file", "songs_ai"])
combined = ai_summaries.merge(df)

In [ ]:
for _, row in combined[:15].iterrows():
    print(f"regex: {row.songs_regex}")
    print(f"AI: {row.songs_ai}")

regex: ["lied: 'samen in de naam van jezus'", "luisterlied: 'boven alle onrust uit'", 'psalm 105:5 (gereformeerd kerkboek)', "lied: 'familie'", 'slotlied: houd vol']
AI: ['Samen in de naam van Jezus ', 'De kerk van alle tijden ', 'boven alle onrust uit ', 'Psalm 105:5 (Gereformeerd Kerkboek)', 'Familie ', 'Houd vol']
